# Part 1: Array-oriented programming

(To be viewed with [jupyterlab-deck](https://jupyterlab-deck.readthedocs.io/).)

## What is "array-oriented programming"?

<br>

It's a programming paradigm, alongside paradigms like "imperative," "object-oriented," and "functional."

<br><br>

**In array-oriented programming, the primary data type is an array, and most functions perform one operation on all the elements of the array.**

<br><br>

In [ ]:
import numpy as np

For instance, this is _not_ array-oriented:

<br>

In [ ]:
input_data = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9])
output_data = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0])

for i in range(len(input_data)):             # explicitly specifies an order of execution
    output_data[i] = input_data[i]**2        # user says what happens to each element

output_data

And this is _not_ array-oriented:

<br>

In [ ]:
input_data = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9])

output_data = np.fromiter(
    map(lambda x: x**2, input_data), int     # still focused on the individual element "x"
)

output_data

_This_ is array-oriented:

<br>

In [ ]:
input_data = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9])

output_data = input_data**2                  # implicit indexes, no individual elements

output_data

As with all programming paradigms, there isn't a sharply defined rule to cleanly separate them, and a single codebase can use several paradigms.

<br>

They are _styles_, and they're useful because they each bring different programming concepts into the foreground:

| Paradigm | Emphasizes |
|:-:|:-:|
| imperative/procedural | low-level algorithms |
| object-oriented | large-scale program structure |
| actor-based | temporal locality |
| literate | human instruction |
| event-driven | cause-and-effect |
| declarative | properties of desired result |
| symbolic | formula transformations |
| functional | data transformations |
| array-oriented | data distributions |

## What is array-oriented programming good for?

<br>

All of the languages/major libraries that emphasize array-oriented programming (that I know of):

<br>

<img src="../img/apl-timeline.svg" width="100%">

Almost all of them are intended as _interactive data-analysis_ environments.

(Fortran-90 is the only one that is not interactive.)

<br><br>

Almost all feature:

* interactive REPL (read-evaluate-print loop)
* concise notation
* unabashedly mathematical

### The grandfather: APL

<table><tr>
    <td width="25%"><img src="../img/apl-book.png" width="100%"></td>
    <td width="50%"><img src="../img/apl-keyboard.jpg" width="100%"></td>
</tr></table>

In [ ]:
%%html
<div style="overflow: hidden;"><iframe src="https://tryapl.org/" width="100%" height="380" scrolling="no" style="border: none;"></div>

**Quizlet:** Translate the following NumPy operations into APL.

<br>

In [ ]:
np.arange(10)

<br>

In [ ]:
np.sum(np.arange(10))

<br>

In [ ]:
np.cumsum(np.arange(10))

In [ ]:
%%html
<!-- This will only work on the day of the live tutorial. -->
<div style="overflow: hidden;"><iframe src="https://app.sli.do/event/rbr8JR3hY4WEZ9CpWm94Xg/embed/polls/d92f941a-23fc-494d-a18b-8163205dc779" width="100%" height="280" scrolling="no" style="border: none;"></div>

**Answers:**

```apl
      (⍳10) - 1
0 1 2 3 4 5 6 7 8 9

      +/(⍳10) - 1
45

      +\(⍳10) - 1
0 1 3 6 10 15 21 28 36 45
```

<br>

APL was too concise! Modern array-oriented programming is looking for the right balance.

<center>
<img src="../img/tshirt.jpg" width="20%">
</center>

### Distributions and interactivity

Array-oriented languages bring data _distributions_ to the foreground.

<br>

In [ ]:
from hist import Hist  # histogram library

<br>

Given a large dataset...

In [ ]:
dataset = np.random.normal(0, 1, 1000000)  # one MILLION data points

<br>

How are the data _distributed_?

In [ ]:
Hist.new.Reg(100, -5, 5).Double().fill(dataset)

What happens if we apply a function to _all values in the distribution_?

In [ ]:
dataset2 = dataset**2

<br>

In [ ]:
Hist.new.Reg(100, -1, 10).Double().fill(dataset2)

Can anyone guess what this distribution will look like?

In [ ]:
dataset3 = np.sin(1/dataset2)

<br>

(I can't.)

In [ ]:
Hist.new.Reg(100, -1, 1).Double().fill(dataset3)

**Human readability advantage:**

  * Mathematical expressions are concise, more convenient to type interactivity.

<br>

**Computational advantage:**

  * The right _part_ of the computation is accelerated: the loop over all values in the distribution.

## NumPy

<img src="../img/Numpy_Python_Cheat_Sheet.svg" width="100%">

NumPy's version of

```apl
      10 20 30 + 1 2 3
11 22 33
```

is

In [ ]:
np.array([10, 20, 30]) + np.array([1, 2, 3])

<br><br>

Since all scalar → scalar operators (and scalar → scalar functions in `np.*`) are promoted to elementwise operations on arrays, any [closed form](https://en.wikipedia.org/wiki/Closed-form_expression) expression duck-types between scalars and arrays.

In [ ]:
def quadratic_formula(a, b, c):
    return (-b + np.sqrt(b**2 - 4*a*c)) / (2*a)

Compute the quadratic formula on one set of scalar values:

In [ ]:
a = 5
b = 10
c = -0.1

quadratic_formula(a, b, c)

<br>

Compute the quadratic formula on a million values in arrays:

In [ ]:
a = np.random.uniform(5, 10, 1000000)
b = np.random.uniform(10, 20, 1000000)
c = np.random.uniform(-0.1, 0.1, 1000000)

quadratic_formula(a, b, c)

### A note about performance

Although NumPy makes each step of the calculation fast, the whole expression is not optimized.

The execution of `quadratic_formula` is _similar to_ this:

In [ ]:
def pedantic_quadratic_formula(a, b, c):
    tmp1 = np.negative(b)            # -b
    tmp2 = np.square(b)              # b**2
    tmp3 = np.multiply(4, a)         # 4*a
    tmp4 = np.multiply(tmp3, c)      # tmp3*c
    del tmp3
    tmp5 = np.subtract(tmp2, tmp4)   # tmp2 - tmp4
    del tmp2, tmp4
    tmp6 = np.sqrt(tmp5)             # sqrt(tmp5)
    del tmp5
    tmp7 = np.add(tmp1, tmp6)        # tmp1 + tmp6
    del tmp1, tmp6
    tmp8 = np.multiply(2, a)         # 2*a
    return np.divide(tmp7, tmp8)     # tmp7 / tmp8

<br>

In [ ]:
pedantic_quadratic_formula(a, b, c)

Although it's faster than Python `for` loops...

In [ ]:
%%timeit

imperative = np.empty_like(c)
for i, (ai, bi, ci) in enumerate(zip(a, b, c)):
    imperative[i] = quadratic_formula(ai, bi, ci)

<br>

In [ ]:
%%timeit

quadratic_formula(a, b, c)

<br>

In [ ]:
%%timeit

pedantic_quadratic_formula(a, b, c)

...it's not as fast as a single "fused" loop over the data.

Other libraries fill this performance gap:

In [ ]:
import numexpr as ne

ne.evaluate("(-b + sqrt(b**2 - 4*a*c)) / (2*a)");

<br>

In [ ]:
import numba as nb

@nb.vectorize
def numba_quadratic_formula(a, b, c):
    return (-b + np.sqrt(b**2 - 4*a*c)) / (2*a)

numba_quadratic_formula(a, b, c);

<br>

In [ ]:
import jax
jax.config.update("jax_platform_name", "cpu"); jax.config.update("jax_enable_x64", True)

@jax.jit
def jax_quadratic_formula(a, b, c):
    return (-b + jax.numpy.sqrt(b**2 - 4*a*c)) / (2*a)

jax_quadratic_formula(a, b, c);

In [ ]:
%%timeit
quadratic_formula(a, b, c)         # NumPy (loop over data for each operation... mostly)

<br>

In [ ]:
%%timeit
ne.re_evaluate()                   # NumExpr (fast virtual machine)

<br>

In [ ]:
%%timeit
numba_quadratic_formula(a, b, c)   # Numba (compiles with LLVM)

<br>

In [ ]:
%%timeit
jax_quadratic_formula(a, b, c)     # JAX (compiles with XLA)

Compiling Python or binding compiled code to Python is an open field, without a clearly dominant solution.

<img src="../img/history-of-bindings-2.svg" width="100%">

### Expressibility in NumPy

The amazing thing is that we can express so much by piecing together precompiled routines.

Combine

  * elementwise operations ("ufuncs")
  * broadcasting
  * basic and advanced slicing
  * slice-assignment
  * reshaping
  * reducers
  * specialized functions

to do great things!

**Example:** Compute the size of the spaces between consecutive elements.

In [ ]:
array = np.array([1.1, 2.2, 3.3, 4.4, 5.5, 6.6, 7.7, 8.8, 9.9])
array

<br><br><br>

In [ ]:
array[1:] - array[:-1]

<center>
<img src="../img/flat-operation.svg" width="70%">
</center>

<center>
<img src="../img/shifted-operation.svg" width="70%">
</center>

**Quizlet:** Compute the length of the curve sampled by arrays `x` and `y`.

In [ ]:
t = np.linspace(0, 2*np.pi, 10000)
x = np.sin(3*t)
y = np.sin(4*t)

<br>

<center>
<img src="../img/length-by-segment.svg" width="55%">
</center>

<br>

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(x, y);

In [ ]:
%%html
<!-- This will only work on the day of the live tutorial. -->
<div style="overflow: hidden;"><iframe src="https://app.sli.do/event/rbr8JR3hY4WEZ9CpWm94Xg/embed/polls/d92f941a-23fc-494d-a18b-8163205dc779" width="100%" height="280" scrolling="no" style="border: none;"></div>

**Answer:**

In [ ]:
np.sum(np.sqrt((x[1:] - x[:-1])**2 + (y[1:] - y[:-1])**2))

**Example:** Big-endian to little-endian conversion.

(Note: `view` and `reshape` do not change the array data, only its interpretation.)

In [ ]:
big_endian = np.array([1.1, 2.2, 3.3, 4.4, 5.5], dtype=">f8")
wrong_endianness = big_endian.view(np.float64)
wrong_endianness

In [ ]:
raw_bytes = wrong_endianness.view(np.uint8)
raw_bytes

In [ ]:
grouped = raw_bytes.reshape(len(raw_bytes) // 8, 8)
grouped

Note: `:` is a full slice (changing nothing) and `::-1` reverses a dimension (`step == -1`).

In [ ]:
reversed_grouped = grouped[:, ::-1]
reversed_grouped

In [ ]:
flattened = reversed_grouped.reshape(len(raw_bytes))
flattened

In [ ]:
flattened.view(wrong_endianness.dtype)

**Quizlet:** Downsample this curve to make it less noisy.

In [ ]:
noisy_data = np.sin(np.linspace(0, 2*np.pi, 1000)) + np.random.normal(0, 0.5, 1000)

<br>

That is, replace every 10 consecutive array elements with their average value, reducing the number of array elements from 1000 to 100, but representing the same curve (ranging from ‒1 to 1).

Note: this is how `axis` works for reducers.

<center>
<img src="../img/example-reducer-2d.svg" width="40%">
</center>

In [ ]:
plt.plot(noisy_data);

In [ ]:
%%html
<!-- This will only work on the day of the live tutorial. -->
<div style="overflow: hidden;"><iframe src="https://app.sli.do/event/rbr8JR3hY4WEZ9CpWm94Xg/embed/polls/d92f941a-23fc-494d-a18b-8163205dc779" width="100%" height="280" scrolling="no" style="border: none;"></div>

**Answer:**

In [ ]:
downsampled = np.mean(noisy_data.reshape(len(noisy_data) // 10, 10), axis=1)
plt.plot(downsampled);

**Go to the [Part 1 exercise](exercise.ipynb) now!**